In [64]:
import os
import glob
import pandas as pd
import datetime
pd.set_option('display.max_columns', 30)
from IPython.display import display
from matplotlib import pyplot as plt

In [54]:
project_dir = os.path.join(".", os.pardir, os.pardir)

#Get processed datasets
filenames = glob.glob(project_dir+"/data/interim/flow_dataset_*.csv")
filenames.sort()

frame = pd.DataFrame()
list_ = []
for file_ in filenames:
    df = pd.read_csv(file_, index_col=0)
    list_.append(df)
df = pd.concat(list_)
df.reset_index(drop=True, inplace=True)
df = df[['SctnId', 'Endereco', 'Direction', 'date', 'hour', 'minute_bin',
         'Velocidade Média (km/h)', 'Atraso médio (s)', 'Fila média (m)', 'Nível médio de congestionamento (0 a 5)', 
         'Numero de faixas', 'Ciclofaixa', 'Corredor','Equipamento',
         '00 a 10', '11 a 20', '21 a 30', '31 a 40', '41 a 50', '51 a 60',
         '61 a 70', '71 a 80', '81 a 90', '91 a 100', 'Acima de 100', 'Total',
        ]]
print(df.shape)
df.sample(5)

(5491, 26)


,SctnId,Endereco,Direction,date,hour,minute_bin,Velocidade Média (km/h),Atraso médio (s),Fila média (m),Nível médio de congestionamento (0 a 5),Numero de faixas,Ciclofaixa,Corredor,Equipamento,00 a 10,11 a 20,21 a 30,31 a 40,41 a 50,51 a 60,61 a 70,71 a 80,81 a 90,91 a 100,Acima de 100,Total
2483,3893.0,Rua Florianópolis x Rua Guanabara,North,2017-11-28,8,15 a 29,10.765000,145.500000,710.5,3.0,1,0,0,FS610JOI,1,68,83,27,0,0,0,0,0,0,0,179
3590,7383.0,Av. José Vieira X Rua Itaiópolis,South,2017-12-12,15,15 a 29,7.576667,134.666667,386.0,3.0,2,0,1,FS596JOI,0,101,177,63,1,0,0,0,0,0,0,342
3535,937.0,Av. José Vieira X Rua Itaiópolis,South,2017-12-13,9,30 a 44,14.230000,155.000000,1047.0,3.0,2,0,1,FS596JOI,0,103,145,87,13,0,0,0,0,0,0,348
1241,3476.0,Av. José Vieira X Rua Itaiópolis,South,2017-11-10,12,15 a 29,9.910000,66.000000,369.0,3.0,2,0,1,FS596JOI,0,105,133,89,6,0,0,0,0,0,0,333
2849,5138.0,Rua Fátima. 542,South,2017-12-04,19,0 a 14,17.455000,98.500000,1168.5,2.5,1,0,0,FS640JOI,0,3,19,65,15,0,0,0,0,0,0,102


Due to buffer approximations, some radars belong simultaneously to more than one street section. For simplicity matters, we will just keep the first section. 

In [63]:
display(df[["Equipamento", "SctnId", "Direction", "date", "hour",
          "minute_bin", "Velocidade Média (km/h)", "Total"]][df["Equipamento"]=="FS596JOI"].\
      sort_values(["date", "hour", "minute_bin"]).tail(10))

df.drop_duplicates(["Equipamento", "Direction", "date", "hour",
          "minute_bin", "Velocidade Média (km/h)"], inplace=True)

df[["Equipamento", "SctnId", "Direction", "date", "hour",
          "minute_bin", "Velocidade Média (km/h)", "Total"]][df["Equipamento"]=="FS596JOI"].\
      sort_values(["date", "hour", "minute_bin"]).tail(10)

,Equipamento,SctnId,Direction,date,hour,minute_bin,Velocidade Média (km/h),Total
4396,FS596JOI,7480.0,South,2017-12-26,9,30 a 44,12.99,240
4393,FS596JOI,7383.0,South,2017-12-26,18,0 a 14,5.78,305
4359,FS596JOI,3476.0,South,2017-12-26,18,15 a 29,10.53,309
4394,FS596JOI,7383.0,South,2017-12-26,18,15 a 29,5.48,309
4360,FS596JOI,3476.0,South,2017-12-26,18,30 a 44,10.92,260
4361,FS596JOI,3476.0,South,2017-12-27,16,45 a 59,9.46,254
4362,FS596JOI,3476.0,South,2017-12-29,19,0 a 14,7.18,210
4363,FS596JOI,3476.0,South,2017-12-29,19,15 a 29,7.14,261
4364,FS596JOI,3476.0,South,2017-12-30,12,15 a 29,5.79,189
4365,FS596JOI,3476.0,South,2017-12-30,21,0 a 14,14.18,185


,Equipamento,SctnId,Direction,date,hour,minute_bin,Velocidade Média (km/h),Total
4396,FS596JOI,7480.0,South,2017-12-26,9,30 a 44,12.99,240
4393,FS596JOI,7383.0,South,2017-12-26,18,0 a 14,5.78,305
4359,FS596JOI,3476.0,South,2017-12-26,18,15 a 29,10.53,309
4394,FS596JOI,7383.0,South,2017-12-26,18,15 a 29,5.48,309
4360,FS596JOI,3476.0,South,2017-12-26,18,30 a 44,10.92,260
4361,FS596JOI,3476.0,South,2017-12-27,16,45 a 59,9.46,254
4362,FS596JOI,3476.0,South,2017-12-29,19,0 a 14,7.18,210
4363,FS596JOI,3476.0,South,2017-12-29,19,15 a 29,7.14,261
4364,FS596JOI,3476.0,South,2017-12-30,12,15 a 29,5.79,189
4365,FS596JOI,3476.0,South,2017-12-30,21,0 a 14,14.18,185
